In [1]:
import pandas as pd
import os
import json


from IPython.display import display

## get files:

In [2]:
questionare_path = "../res/user_data/questionare.xlsx"
participant_order_path = "../res/participant_order.xlsx"
user_data_path = "../res/user_data/"

chat_bot_files  = [f for f in os.listdir(user_data_path) if f.endswith('.json')]

df_questionare = pd.read_excel(open(questionare_path, 'rb'), index_col=0)
df_participant_order = pd.read_excel(open(participant_order_path, 'rb'), index_col=0)



## questionare data preparation:

In [3]:
question_labels = ["The chatbot’s personality was realistic and engaging",
                   "The chatbot seemed too robotic",
                   "The chatbot was welcoming during initial setup",
                   "The chatbot seemed very unfriendly",
                   "The chatbot explained its scope and purpose well",
                   "The chatbot gave no indication as to its purpose",
                   "The chatbot was easy to navigate","It would be easy to get confused when using the chatbot",
                   "The chatbot understood me well","The chatbot failed to recognise a lot of my inputs",
                   "Chatbot responses were useful, appropriate and informative","Chatbot responses were irrelevant",
                   "The chatbot coped well with any errors or mistakes",
                   "The chatbot seemed unable to handle any errors",
                   "The chatbot was very easy to use",
                   "The chatbot was very complex"
                   ]
question_labels_2 = [x + "2" for x in question_labels]
rename_dict = {v: k for k, v in zip(question_labels, question_labels_2)}



In [4]:
df = df_questionare.merge(df_participant_order, left_on="Participant code", right_on="Participant number")
df.drop(columns=["Please complete the first three trials", "Please complete the last three trials", 'Task 1', 'Task 2', 'Task 3', 'Task 4', 'Task 5', 'Task 6'], inplace=True)
df.drop(columns=["​Experiment Description\n\nThe aim of the project is to determine how the communication style of a chatbot impacts the perceived quality of it. The communication styles vary on formality of language..."], inplace=True)
#print(df.columns)

In [5]:
# create df for genz
df_questionare_genz_1 = df[df["Chatbot group start"].str.match("Informal")]
df_questionare_genz_1 = df_questionare_genz_1.drop(columns=question_labels_2)
df_questionare_genz_2 = df[df["Chatbot group start"].str.match("Formal")]
df_questionare_genz_2 = df_questionare_genz_2.drop(columns=question_labels)
df_questionare_genz_2  = df_questionare_genz_2.rename(columns=rename_dict)


df_questionare_genz = pd.concat([df_questionare_genz_1, df_questionare_genz_2 ])
df_questionare_genz["language"] = "genz"
print(len(df_questionare_genz)) 


4


In [6]:
# same as above get df for formal
df_questionare_formal_1 = df[df["Chatbot group start"].str.match("Formal")]
df_questionare_formal_1 = df_questionare_formal_1.drop(columns=question_labels_2)
df_questionare_formal_2 = df[df["Chatbot group start"].str.match("Informal")]
df_questionare_formal_2 = df_questionare_formal_2.drop(columns=question_labels)
df_questionare_formal_2 = df_questionare_formal_2.rename(columns=rename_dict)

df_questionare_formal = pd.concat([df_questionare_formal_1, df_questionare_formal_2 ])
df_questionare_formal = df_questionare_formal.drop(columns=["Chatbot group start"])
df_questionare_formal["language"] = "formal"
#print((df_questionare_formal.columns)) 

In [7]:
df_questionare = pd.concat([df_questionare_genz, df_questionare_formal])
df_questionare = df_questionare.set_index(["Participant code", "language"])
df_questionare = df_questionare.drop(columns=["Chatbot group start"])


In [8]:
# 'Chatbot group start' gives the order of the user did the language options

In [9]:
def calc_CUQ_score(df):
    # step 1:
    agree_scores = {
        "Strongly agree": 5,
        "Agree": 4,
        "Neutral": 3,
        "Disagree": 2,
        "Strongly disagree": 1
    }
    # step 2 and 3:
    # odd and even are swapped, cause we count from 0
    # step 2
    positiv_score = 0
    for i in range(0,len(question_labels), 2):
        positiv_score += agree_scores[df[question_labels[i]]]
    # step 3:
    negative_score = 0
    for i in range(1,len(question_labels), 2):
        negative_score += agree_scores[df[question_labels[i]]]
    
    #step 4:
    positiv_score = positiv_score - 8
    # step 5:
    negative_score = 40 - negative_score
    # step 6:
    cuq_score = positiv_score + negative_score 
    #You should now have a score out of 64 
    if cuq_score > 64:
        raise ValueError("CUQ score is greater than 64")
    # step 7:
    cuq_score = cuq_score / 64 
    cuq_score = cuq_score * 100
    return cuq_score


for index, row in df_questionare.iterrows():
    print( calc_CUQ_score(row))
    df_questionare.at[index, "CUQ"] = calc_CUQ_score(row)




37.5
46.875
70.3125
65.625
62.5
40.625
51.5625
53.125


In [10]:
print("Questionare Dataset:")
display(df_questionare)

Questionare Dataset:


,,Start time,Completion time,Email,Name,Last modified time,Age,The chatbot’s personality was realistic and engaging,The chatbot seemed too robotic,The chatbot was welcoming during initial setup,The chatbot seemed very unfriendly,...,It would be easy to get confused when using the chatbot,The chatbot understood me well,The chatbot failed to recognise a lot of my inputs,"Chatbot responses were useful, appropriate and informative",Chatbot responses were irrelevant,The chatbot coped well with any errors or mistakes,The chatbot seemed unable to handle any errors,The chatbot was very easy to use,The chatbot was very complex,CUQ
Participant code,language,,,,,,,,,,,,,,,,,,,,,
6,genz,2023-10-24 14:32:05,2023-10-24 14:47:12,anonymous,NaN,NaN,23,Agree,Strongly agree,Agree,Strongly disagree,...,Strongly agree,Disagree,Strongly agree,Neutral,Agree,Strongly disagree,Strongly agree,Disagree,Disagree,37.5000
2,genz,2023-10-25 17:42:31,2023-10-25 17:55:19,anonymous,NaN,NaN,28,Neutral,Agree,Neutral,Disagree,...,Agree,Disagree,Neutral,Agree,Disagree,Strongly disagree,Neutral,Neutral,Disagree,46.8750
25,genz,2023-10-24 16:25:52,2023-10-24 17:05:53,anonymous,NaN,NaN,25,Strongly agree,Strongly disagree,Strongly agree,Strongly disagree,...,Agree,Agree,Neutral,Agree,Disagree,Disagree,Disagree,Agree,Agree,70.3125
1,genz,2023-10-24 21:38:57,2023-10-24 21:52:38,anonymous,NaN,NaN,24,Agree,Disagree,Agree,Strongly disagree,...,Agree,Agree,Neutral,Agree,Disagree,Neutral,Neutral,Agree,Neutral,65.6250
25,formal,2023-10-24 16:25:52,2023-10-24 17:05:53,anonymous,NaN,NaN,25,Agree,Disagree,Strongly agree,Neutral,...,Agree,Disagree,Agree,Agree,Strongly disagree,Disagree,Disagree,Agree,Agree,62.5000
1,formal,2023-10-24 21:38:57,2023-10-24 21:52:38,anonymous,NaN,NaN,24,Neutral,Agree,Strongly agree,Disagree,...,Strongly agree,Disagree,Agree,Neutral,Neutral,Disagree,Agree,Disagree,Neutral,40.6250
6,formal,2023-10-24 14:32:05,2023-10-24 14:47:12,anonymous,NaN,NaN,23,Disagree,Strongly agree,Neutral,Strongly disagree,...,Agree,Disagree,Agree,Agree,Disagree,Disagree,Neutral,Agree,Disagree,51.5625
2,formal,2023-10-25 17:42:31,2023-10-25 17:55:19,anonymous,NaN,NaN,28,Agree,Neutral,Agree,Strongly disagree,...,Agree,Neutral,Neutral,Agree,Strongly disagree,Disagree,Agree,Disagree,Disagree,53.1250


In [11]:
# user id

# total time (average of tasks with same language)
# number of suggestions (average of tasks with same language)
# number of turns (average of tasks with same language)
# number_of_suggestions
# number_of_restarts
df_chat = pd.DataFrame(columns=["user_id",  "language",  "number_of_turns", "time", "number_of_suggestions", "number_of_restarts"])
for chat_data in chat_bot_files:
    chat_data = json.load(open(user_data_path + chat_data))
    row = df_chat.loc[(df_chat["user_id"] == chat_data["user_id"] ) & (df_chat["language"] == chat_data["configuration"]["language"]  ) ]
    total_time = sum([x["response_seconds"] for x in chat_data["turns"]])
    if row.empty:

        row = pd.DataFrame([[ chat_data["user_id"],  chat_data["configuration"]["language"],  len(chat_data["turns"]) , total_time, chat_data["number_suggestions"], chat_data["number_restart"] ] ], 
                           columns=["user_id",  "language", "number_of_turns", "time", "number_of_suggestions", "number_of_restarts"])

        df_chat = pd.concat([df_chat, row])
    else:
        row.loc[0,"number_of_turns"] = row.loc[0,"number_of_turns"] + len(chat_data["turns"])
        row.loc[0,"time"] = row.loc[0,"time"] + total_time
        row.loc[0,"number_of_suggestions"] = row.loc[0,"number_of_suggestions"] + chat_data["number_suggestions"]
        row.loc[0, "number_of_restarts"] = row.loc[0, "number_of_restarts"] + chat_data["number_restart"]
        df_chat.loc[(df_chat["user_id"] == chat_data["user_id"] ) & (df_chat["language"] == chat_data["configuration"]["language"]  ) ] = row

df_chat["number_of_turns"] = df_chat["number_of_turns"] / 3
df_chat["time"] = df_chat["time"] / 3
df_chat["number_of_suggestions"] = df_chat["number_of_suggestions"] / 3
df_chat["number_of_restarts"] = df_chat["number_of_restarts"] / 3



df_chat["language"] = df_chat['language'].str.lower()
#df_chat.rename(columns={"user_id": "Participant code"}, inplace=True)
df_chat.set_index(["user_id", "language"], inplace=True)
print("Chatbot Dataframe:")
display(df_chat)

Chatbot Dataframe:


number_of_turns        time number_of_suggestions  \
user_id language                                                     
6       genz           12.333333   83.626476                   1.0   
        formal          9.666667   80.554383                   1.0   
1       formal              13.0  370.978024              0.666667   
2       genz                 8.0   94.155803                   0.0   
        formal          9.333333   79.188921                   1.0   
1       genz                 8.0  125.788085                   0.0   

                 number_of_restarts  
user_id language                     
6       genz               1.333333  
        formal             0.333333  
1       formal             0.666667  
2       genz               0.666667  
        formal             0.333333  
1       genz                    0.0

In [12]:
df_chat.reset_index(inplace=True)
df_questionare.reset_index(inplace=True)

In [70]:
### the index -unique identifier- of df is (participant code, language)
df_questionare["index"] = df_questionare["Participant code"].astype(str) + df_questionare["language"]
df_chat["index"] = df_chat["user_id"] + df_chat["language"]
df = df_questionare.merge(df_chat, left_on=["index"], right_on=["index"])
df = df.drop(columns=["index", "language_y", "user_id"] )# deleting other user id, so we dont need to reorder to have it at the first position..
df = df.rename(columns={"language_x": "language", "Participant code": "user_id"})
#df.reset_index (inplace=True)  # cause idk pandas i cant work with this index
#df.set_index(["Participant code", "language"], inplace=True)
print("Merged Dataframe:")
display(df)


Merged Dataframe:


,user_id,language,Start time,Completion time,Email,Name,Last modified time,Age,The chatbot’s personality was realistic and engaging,The chatbot seemed too robotic,...,Chatbot responses were irrelevant,The chatbot coped well with any errors or mistakes,The chatbot seemed unable to handle any errors,The chatbot was very easy to use,The chatbot was very complex,CUQ,number_of_turns,time,number_of_suggestions,number_of_restarts
0,6,genz,2023-10-24 14:32:05,2023-10-24 14:47:12,anonymous,NaN,NaN,23,Agree,Strongly agree,...,Agree,Strongly disagree,Strongly agree,Disagree,Disagree,37.5000,12.333333,83.626476,1.0,1.333333
1,2,genz,2023-10-25 17:42:31,2023-10-25 17:55:19,anonymous,NaN,NaN,28,Neutral,Agree,...,Disagree,Strongly disagree,Neutral,Neutral,Disagree,46.8750,8.0,94.155803,0.0,0.666667
2,1,genz,2023-10-24 21:38:57,2023-10-24 21:52:38,anonymous,NaN,NaN,24,Agree,Disagree,...,Disagree,Neutral,Neutral,Agree,Neutral,65.6250,8.0,125.788085,0.0,0.0
3,1,formal,2023-10-24 21:38:57,2023-10-24 21:52:38,anonymous,NaN,NaN,24,Neutral,Agree,...,Neutral,Disagree,Agree,Disagree,Neutral,40.6250,13.0,370.978024,0.666667,0.666667
4,6,formal,2023-10-24 14:32:05,2023-10-24 14:47:12,anonymous,NaN,NaN,23,Disagree,Strongly agree,...,Disagree,Disagree,Neutral,Agree,Disagree,51.5625,9.666667,80.554383,1.0,0.333333
5,2,formal,2023-10-25 17:42:31,2023-10-25 17:55:19,anonymous,NaN,NaN,28,Agree,Neutral,...,Strongly disagree,Disagree,Agree,Disagree,Disagree,53.1250,9.333333,79.188921,1.0,0.333333


In [115]:
import matplotlib.pyplot as plt
cuq_formal = df[df["language"].str.match("formal")]["CUQ"]

cuq_genz = df[df["language"].str.match("genz")]["CUQ"]
display(cuq_genz)
df = pd.DataFrame({'A':cuq_genz,
                   'B':cuq_formal})
df = pd.DataFrame({('genz': cuq_genz , dtype='float',
                   'formal':cuq_formal, dtype='float')})
cuq_plot = pd.DataFrame( [],columns=["formal", "genz"], dtype= [pd.np.int64,pd.np.int64])
cuq_plot["genz"] = cuq_genz
cuq_plot["formal"] = cuq_formal
#df_plot = df[df["language"].str.match("formal")][]

#print (cuq_formal.columns)
#df_plot = df_plot.set_index(["user_id", "language"])
display(cuq_plot)

cuq_plot.plot.box()

#cuq_genz.plot()
#df.plot()

0    37.500
1    46.875
2    65.625
Name: CUQ, dtype: float64

AttributeError: module 'pandas' has no attribute 'np'